In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the dataset
data_path = r"time_New_FFFinal_formatted_Figi.csv"
data = pd.read_csv(data_path, parse_dates=['time_series'], index_col='time_series')

# Select the target variable
sea_level = data['Sea Level']



# Split data into training and test sets
train_size = int(len(sea_level) * 0.8)
train, test = sea_level[:train_size], sea_level[train_size:]

# LSTM Model
def lstm_model(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train.values.reshape(-1, 1))
    scaled_test = scaler.transform(test.values.reshape(-1, 1))

    # Prepare the data for LSTM
    X_train, y_train = [], []
    for i in range(60, len(scaled_train)):
        X_train.append(scaled_train[i-60:i])
        y_train.append(scaled_train[i])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshape for LSTM [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Building the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs=1)

    # Make predictions
    predictions = model.predict(X_train) # Changed to X_train here for prediction 
    
    # Inverse transform to get actual sea level values
    lstm_predictions = scaler.inverse_transform(predictions)
    

    # Calculate MAE and RMSE
    lstm_mae = mean_absolute_error(y_train, predictions) # Changed to y_train, predictions for MAE calculation
    lstm_rmse = np.sqrt(mean_squared_error(y_train, predictions)) # Changed to y_train, predictions for RMSE calculation


    return lstm_predictions, lstm_mae, lstm_rmse # Added return statement


# Run LSTM model
lstm_predictions, lstm_mae, lstm_rmse = lstm_model(train, test)

# Plotting actual vs predicted values
plt.figure(figsize=(14, 7))
plt.plot(test.index, test, label='Actual Sea Level', color='blue')
# Use the training data index for plotting the predictions, since that's what the model was trained on
plt.plot(train.index[60:len(lstm_predictions)+60], lstm_predictions, label='LSTM Predictions', color='red')  
plt.title('Sea Level Predictions vs Actual (LSTM)')
plt.xlabel('Date')
plt.ylabel('Sea Level')
plt.legend()
plt.show()

# Print accuracy of the LSTM model at the end
print("LSTM MAE:", lstm_mae)
print("LSTM RMSE:", lstm_rmse)